### Import necessary dependencies

In [1]:
!pip install setuptools




Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
pip install --upgrade pandas

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
from packaging.version import Version as LooseVersion
from pyspark.sql import SparkSession
from pyspark.sql.functions import monotonically_increasing_id 
import pandas as pd
from datetime import datetime, timedelta
from sqlalchemy import create_engine
import psycopg2

In [4]:
Spark

NameError: name 'Spark' is not defined

In [ ]:
spark = SparkSession.builder.appName("NUGABANKETL").getOrCreate()

### Data Extraction

In [92]:
nuga_bank_df = spark.read.csv(r'nuga_bank_transactions.csv',header = True)

In [93]:
nuga_bank_df.show(10)

+--------------------+------+----------------+--------------+--------------------+--------------------+--------------+--------------------+--------------------+--------------------+--------------------+-------------------+-------------------+--------------------+-------------+-------------+--------+-----+---------+--------------------+--------------------+------+--------------+
|    Transaction_Date|Amount|Transaction_Type| Customer_Name|    Customer_Address|       Customer_City|Customer_State|    Customer_Country|             Company|           Job_Title|               Email|       Phone_Number| Credit_Card_Number|                IBAN|Currency_Code|Random_Number|Category|Group|Is_Active|        Last_Updated|         Description|Gender|Marital_Status|
+--------------------+------+----------------+--------------+--------------------+--------------------+--------------+--------------------+--------------------+--------------------+--------------------+-------------------+----------------

In [94]:
nuga_bank_df.printSchema()

root
 |-- Transaction_Date: string (nullable = true)
 |-- Amount: string (nullable = true)
 |-- Transaction_Type: string (nullable = true)
 |-- Customer_Name: string (nullable = true)
 |-- Customer_Address: string (nullable = true)
 |-- Customer_City: string (nullable = true)
 |-- Customer_State: string (nullable = true)
 |-- Customer_Country: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Job_Title: string (nullable = true)
 |-- Email: string (nullable = true)
 |-- Phone_Number: string (nullable = true)
 |-- Credit_Card_Number: string (nullable = true)
 |-- IBAN: string (nullable = true)
 |-- Currency_Code: string (nullable = true)
 |-- Random_Number: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Group: string (nullable = true)
 |-- Is_Active: string (nullable = true)
 |-- Last_Updated: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Marital_Status: string (nullable = true)



In [95]:
## to read your file and also show the correct datatype use this code,
nuga_bank_df = spark.read.csv(r'nuga_bank_transactions.csv',header = True, inferSchema=True)

In [96]:
nuga_bank_df.printSchema()

root
 |-- Transaction_Date: timestamp (nullable = true)
 |-- Amount: double (nullable = true)
 |-- Transaction_Type: string (nullable = true)
 |-- Customer_Name: string (nullable = true)
 |-- Customer_Address: string (nullable = true)
 |-- Customer_City: string (nullable = true)
 |-- Customer_State: string (nullable = true)
 |-- Customer_Country: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Job_Title: string (nullable = true)
 |-- Email: string (nullable = true)
 |-- Phone_Number: string (nullable = true)
 |-- Credit_Card_Number: long (nullable = true)
 |-- IBAN: string (nullable = true)
 |-- Currency_Code: string (nullable = true)
 |-- Random_Number: double (nullable = true)
 |-- Category: string (nullable = true)
 |-- Group: string (nullable = true)
 |-- Is_Active: string (nullable = true)
 |-- Last_Updated: timestamp (nullable = true)
 |-- Description: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Marital_Status: string (nullable = true)

### Data Cleaning and Transformation

In [97]:
# check your columns
nuga_bank_df.columns

['Transaction_Date',
 'Amount',
 'Transaction_Type',
 'Customer_Name',
 'Customer_Address',
 'Customer_City',
 'Customer_State',
 'Customer_Country',
 'Company',
 'Job_Title',
 'Email',
 'Phone_Number',
 'Credit_Card_Number',
 'IBAN',
 'Currency_Code',
 'Random_Number',
 'Category',
 'Group',
 'Is_Active',
 'Last_Updated',
 'Description',
 'Gender',
 'Marital_Status']

In [98]:
# number of rows
num_rows = nuga_bank_df.count()

num_rows

1000000

In [99]:
# number of columns
num_columns = len(nuga_bank_df.columns)
num_columns

23

In [100]:
# checking for null values
for columns in nuga_bank_df.columns:
        print(columns, 'nulls', nuga_bank_df.filter(nuga_bank_df[columns].isNull()).count())

Transaction_Date nulls 0
Amount nulls 0
Transaction_Type nulls 0
Customer_Name nulls 100425
Customer_Address nulls 100087
Customer_City nulls 100034
Customer_State nulls 100009
Customer_Country nulls 100672
Company nulls 100295
Job_Title nulls 99924
Email nulls 100043
Phone_Number nulls 100524
Credit_Card_Number nulls 100085
IBAN nulls 100300
Currency_Code nulls 99342
Random_Number nulls 99913
Category nulls 100332
Group nulls 100209
Is_Active nulls 100259
Last_Updated nulls 100321
Description nulls 100403
Gender nulls 99767
Marital_Status nulls 99904


In [101]:
# How to fill up missing or null values
nuga_bank_df_clean = nuga_bank_df.fillna({
    'Customer_Name': 'unknown',
    'Customer_Address': 'unknown',
    'Customer_City': 'unknown',
    'Customer_State':'unknown',
    'Customer_Country': 'unknown',
    'Company': 'unknown',
    'Job_Title': 'unknown',
    'Email': 'unknown',
    'Phone_Number': 'unknown',
    'Credit_Card_Number': 0,
    'IBAN': 'unknown',
    'Currency_Code': 'unknown',
    'Random_Number': 0.0,
    'Category': 'unknown',
    'Group': 'unknown',
    'Is_Active': 'unknown',
    'Description': 'unknown',
    'Gender': 'unknown',
    'Marital_Status': 'unknown'
})

In [102]:
# Drop rows where last updated is null
nuga_bank_df_clean = nuga_bank_df_clean.na.drop(subset=['Last_Updated'])

In [103]:
# confirm changes made for null values
for columns in nuga_bank_df_clean.columns:
        print(columns, 'nulls', nuga_bank_df_clean.filter(nuga_bank_df_clean[columns].isNull()).count())

Transaction_Date nulls 0
Amount nulls 0
Transaction_Type nulls 0
Customer_Name nulls 0
Customer_Address nulls 0
Customer_City nulls 0
Customer_State nulls 0
Customer_Country nulls 0
Company nulls 0
Job_Title nulls 0
Email nulls 0
Phone_Number nulls 0
Credit_Card_Number nulls 0
IBAN nulls 0
Currency_Code nulls 0
Random_Number nulls 0
Category nulls 0
Group nulls 0
Is_Active nulls 0
Last_Updated nulls 0
Description nulls 0
Gender nulls 0
Marital_Status nulls 0


In [104]:
# confirm changes made for drop command.
num_rows = nuga_bank_df_clean.count()

num_rows

899679

In [105]:
# To have an overview of summary statistics of the data
nuga_bank_df_clean.describe().show()

+-------+------------------+----------------+-------------+--------------------+-------------+--------------+----------------+------------+------------------+-------------------+--------------------+--------------------+--------------------+-------------+-----------------+--------+-------+---------+--------------------+-------+--------------+
|summary|            Amount|Transaction_Type|Customer_Name|    Customer_Address|Customer_City|Customer_State|Customer_Country|     Company|         Job_Title|              Email|        Phone_Number|  Credit_Card_Number|                IBAN|Currency_Code|    Random_Number|Category|  Group|Is_Active|         Description| Gender|Marital_Status|
+-------+------------------+----------------+-------------+--------------------+-------------+--------------+----------------+------------+------------------+-------------------+--------------------+--------------------+--------------------+-------------+-----------------+--------+-------+---------+----------

In [106]:
# create a data model using the appropriate tool ( lucid or draw io)


In [107]:
nuga_bank_df_clean.columns

['Transaction_Date',
 'Amount',
 'Transaction_Type',
 'Customer_Name',
 'Customer_Address',
 'Customer_City',
 'Customer_State',
 'Customer_Country',
 'Company',
 'Job_Title',
 'Email',
 'Phone_Number',
 'Credit_Card_Number',
 'IBAN',
 'Currency_Code',
 'Random_Number',
 'Category',
 'Group',
 'Is_Active',
 'Last_Updated',
 'Description',
 'Gender',
 'Marital_Status']

### Table creation

In [108]:
# Transactaction table
Transaction = nuga_bank_df_clean.select('Transaction_Date', 'Amount', 'Transaction_Type')
# Add the transaction_ID column
Transaction = Transaction.withColumn('Transaction_ID', monotonically_increasing_id())
# Reordering columns to make sure the added column comes first
Transaction = Transaction.select('Transaction_ID','Transaction_Date', 'Amount', 'Transaction_Type')

Transaction.show()

+--------------+--------------------+------+----------------+
|Transaction_ID|    Transaction_Date|Amount|Transaction_Type|
+--------------+--------------------+------+----------------+
|             0|2024-03-23 15:38:...| 34.76|      Withdrawal|
|             1|2024-04-22 19:15:...|163.92|      Withdrawal|
|             2|2024-04-12 19:46:...|386.32|      Withdrawal|
|             3|2024-04-17 15:29:...|407.15|         Deposit|
|             4|2024-02-10 01:51:...|161.31|         Deposit|
|             5|2024-02-10 22:56:...|764.34|        Transfer|
|             6|2024-04-07 00:07:...|734.59|         Deposit|
|             7|2024-03-08 01:51:...|592.43|         Deposit|
|             8|2024-02-01 12:34:...| 927.1|         Deposit|
|             9|2024-03-22 16:46:...| 66.59|        Transfer|
|            10|2024-04-23 13:30:...| 246.3|      Withdrawal|
|            11|2024-01-13 01:22:...|782.32|      Withdrawal|
|            12|2024-02-25 15:16:...|818.42|      Withdrawal|
|       

In [109]:
# Customer table
# to reduce reductancy or repeatation where one customer made multiple purchase use (.distinct)
Customer = nuga_bank_df_clean.select('Customer_Name','Customer_Address','Customer_City',
                                        'Customer_State','Customer_Country','Email','Phone_Number').distinct()
# Add the Customer_ID column
Customer = Customer.withColumn('Customer_ID', monotonically_increasing_id())
# Reordering columns to make sure the added column comes first
Customer = Customer.select('Customer_ID','Customer_Name','Customer_Address','Customer_City',
                                        'Customer_State','Customer_Country','Email','Phone_Number')

Customer.show()

+-----------+------------------+--------------------+--------------------+--------------+--------------------+--------------------+--------------------+
|Customer_ID|     Customer_Name|    Customer_Address|       Customer_City|Customer_State|    Customer_Country|               Email|        Phone_Number|
+-----------+------------------+--------------------+--------------------+--------------+--------------------+--------------------+--------------------+
|          0|    Allen Castillo|   5750 Vanessa Neck|     New Vickiemouth|North Carolina|              Zambia|             unknown|  732.974.7438x89666|
|          1|        Tina Jones|28150 Kelsey Stat...|             unknown|          Iowa|               Qatar|gabriellemoore@ex...|    736.645.3977x275|
|          2|    Michael Murphy|894 Williams Ridg...|       Dominguezview|      New York|              Sweden|kristinstanley@ex...|+1-693-739-2204x8851|
|          3|       Brian Glenn|505 Mcdowell Gard...|South Christinech...|  South 

In [ ]:
# Employee table
Employee = nuga_bank_df_clean.select('Company','Job_Title','Gender','Marital_Status').distinct()
# Add the Customer_ID column
Employee = Employee.withColumn('Employee_ID', monotonically_increasing_id())
# Reordering columns to make sure the added column comes first
Employee = Employee.select('Employee_ID','Company','Job_Title','Gender','Marital_Status')

Employee.show()

In [111]:
# Fact table
Fact_table = nuga_bank_df.join(Customer, ['Customer_Name','Customer_Address','Customer_City',\
                                        'Customer_State','Customer_Country','Email','Phone_Number'], 'left')\
                         .join(Transaction, ['Transaction_Date', 'Amount', 'Transaction_Type'],'left')\
                         .join(Employee, ['Company','Job_Title','Gender','Marital_Status'], 'left')\
                         .select('Transaction_ID','Customer_ID','Employee_ID','Credit_Card_Number','IBAN',\
                                 'Currency_Code','Random_Number','Category','Group','Is_Active','Last_Updated','Description',)   


Fact_table.show()


+--------------+-----------+-----------+------------------+--------------------+-------------+-------------+--------+-----+---------+--------------------+--------------------+
|Transaction_ID|Customer_ID|Employee_ID|Credit_Card_Number|                IBAN|Currency_Code|Random_Number|Category|Group|Is_Active|        Last_Updated|         Description|
+--------------+-----------+-----------+------------------+--------------------+-------------+-------------+--------+-----+---------+--------------------+--------------------+
|   25769803789|     198798|       NULL|  6011153251503935|GB28FTRX421798999...|          CHF|       1792.0|       C|    Z|      Yes|2020-07-14 19:25:...|Compare sell bank...|
|             1|25769857338|       NULL|              NULL|GB03KFZR339662263...|          VEF|       2122.0|       B|    Z|     NULL|2020-12-27 13:23:...|Teach edge make n...|
|          NULL|       NULL|       NULL|  4263644221766495|GB47BGNG208602249...|          AOA|       6148.0|       D|   

In [80]:
# output or save transformed data as csv file
#Transaction.repartition(1).write.mode('overwrite').option('header', 'True').csv(r'C:\Users\back2\Desktop\(PYSPARK case_study)/Transaction')
#Employee.repartition(1).write.mode('overwrite').option('header', 'True').csv(r'C:\Users\back2\Desktop\(PYSPARK case_study)/Employee')
#Customer.repartition(1).write.mode('overwrite').option('header', 'True').csv(r'C:\Users\back2\Desktop\(PYSPARK case_study)/Customer')
#Fact_table.repartition(1).write.mode('overwrite').option('header', 'True').csv(r'C:\Users\back2\Desktop\(PYSPARK case_study)/Fact_table')


In [ ]:
# convert spark_df to pandas_df
Transaction_pd_df = Transaction.toPandas()
Customer_pd_df = Customer.toPandas()
Employee_pd_df = Employee.toPandas()
Fact_table_pd_df = Fact_table.toPandas()


In [ ]:
# Define database parameters including the database name
db_params ={
    'username': 'postgres',
    'password': '0852',
    'host': 'localhost',
    'port': '5432',
    'database': 'nuga_bank'
}
db_url = f"postgresql://{db_params['username']}:{db_params['password']}@{db_params['host']}:{db_params['port']}/{db_params['database']}"
# create the database engine with the db url
engine = create_engine(db_url)
# connect to PostgreSQL server
with engine.connect() as connection:

# create table and load the data
    Transaction_pd_df.to_sql('Transaction', connection, index=False, if_exists='replace')
    Customer_pd_df.to_sql('Customer', connection, index=False, if_exists='replace')
    Employee_pd_df.to_sql('Employee', connection, index=False, if_exists='replace')
    Fact_table_pd_df.to_sql('Fact_table', connection, index=False, if_exists='replace')
print('successfull')